In [1]:
pip install -U datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.9/170.9 kB 10.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.3.0
    Uninstalling fsspec-2024.3.0:
      Successfully uninstalled fsspec-2024.3.0
  Attempting uninstall: datasets
    Found existing installation: datasets 2.1.0
    Uninstalling datasets-2.1.0:
      Successfully uninstalled datasets-2.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2023.12.2.post1 requires fsspec==2023.12.2, but you have fsspec 2024.2.0 which is incompatible.
s3fs 2024.3.0 requires fsspec==2024.3.0, but you have fsspec 2024.2.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from datasets import load_dataset , Dataset, concatenate_datasets 
import numpy as np
import pandas as pd
import random

In [3]:
text_to_sql = load_dataset("b-mc2/sql-create-context")
txsql_df = pd.DataFrame(text_to_sql)
display(txsql_df)

Generating train split: 0 examples [00:00, ? examples/s]

,train
0,{'answer': 'SELECT COUNT(*) FROM head WHERE ag...
1,"{'answer': 'SELECT name, born_state, age FROM ..."
2,"{'answer': 'SELECT creation, name, budget_in_b..."
3,"{'answer': 'SELECT MAX(budget_in_billions), MI..."
4,{'answer': 'SELECT AVG(num_employees) FROM dep...
...,...
78572,{'answer': 'SELECT time FROM table_name_35 WHE...
78573,{'answer': 'SELECT ground FROM table_name_83 W...
78574,{'answer': 'SELECT competition FROM table_name...
78575,{'answer': 'SELECT COUNT(decile) FROM table_na...


In [4]:
import json 
dsdf_ = pd.json_normalize(txsql_df['train'])
#Dropping all examples where no definite table name is given i.e. most of WikiSQL
dsdf_ = dsdf_[~dsdf_['answer'].str.contains('FROM table_')]
dsdf_.drop_duplicates(inplace=True)
display(dsdf_), dsdf_.shape
     

,answer,question,context
0,SELECT COUNT(*) FROM head WHERE age > 56,How many heads of the departments are older th...,CREATE TABLE head (age INTEGER)
1,"SELECT name, born_state, age FROM head ORDER B...","List the name, born state and age of the heads...","CREATE TABLE head (name VARCHAR, born_state VA..."
2,"SELECT creation, name, budget_in_billions FROM...","List the creation year, name and budget of eac...","CREATE TABLE department (creation VARCHAR, nam..."
3,"SELECT MAX(budget_in_billions), MIN(budget_in_...",What are the maximum and minimum budget of the...,CREATE TABLE department (budget_in_billions IN...
4,SELECT AVG(num_employees) FROM department WHER...,What is the average number of employees of the...,CREATE TABLE department (num_employees INTEGER...
...,...,...,...
39336,SELECT MIN(league_cup) FROM table,What is the lowest league cup?,CREATE TABLE table (league_cup INTEGER)
39337,SELECT position FROM table WHERE player = Becchio,What is the position for the player becchio?,"CREATE TABLE table (position VARCHAR, player V..."
39338,SELECT COUNT(league_cup) FROM table WHERE r = 4,How many times was the r 4?,"CREATE TABLE table (league_cup VARCHAR, r VARC..."
59542,"SELECT college FROM ""t"" AS able_name_20 WHERE ...",What college did the T who was pick after 13 g...,CREATE TABLE t (Id VARCHAR)


(None, (4540, 3))

In [5]:
dsdf_['question'] = 'Generate SQL query: '+ dsdf_['question']
dsdf_['context'] = dsdf_['context'].apply(lambda x: x.replace("CREATE TABLE", "").replace("VARCHAR", "STRING").strip())
dsdf_['Instruction'] = dsdf_['question']+ ', given the following schema: ' + dsdf_['context']
dsdf_.drop(columns=['question', 'context'], inplace=True)
display(dsdf_)
     


,answer,Instruction
0,SELECT COUNT(*) FROM head WHERE age > 56,Generate SQL query: How many heads of the depa...
1,"SELECT name, born_state, age FROM head ORDER B...","Generate SQL query: List the name, born state ..."
2,"SELECT creation, name, budget_in_billions FROM...","Generate SQL query: List the creation year, na..."
3,"SELECT MAX(budget_in_billions), MIN(budget_in_...",Generate SQL query: What are the maximum and m...
4,SELECT AVG(num_employees) FROM department WHER...,Generate SQL query: What is the average number...
...,...,...
39336,SELECT MIN(league_cup) FROM table,Generate SQL query: What is the lowest league ...
39337,SELECT position FROM table WHERE player = Becchio,Generate SQL query: What is the position for t...
39338,SELECT COUNT(league_cup) FROM table WHERE r = 4,Generate SQL query: How many times was the r 4...
59542,"SELECT college FROM ""t"" AS able_name_20 WHERE ...",Generate SQL query: What college did the T who...


In [6]:
dsdf_[dsdf_['answer'].str.contains('JOIN')].iloc[6]['answer']

'SELECT T1.Official_Name FROM city AS T1 JOIN farm_competition AS T2 ON T1.City_ID = T2.Host_city_ID GROUP BY T2.Host_city_ID HAVING COUNT(*) > 1'

In [7]:
dsdf_[dsdf_['answer'].str.contains('JOIN')]

,answer,Instruction
6,SELECT DISTINCT T1.creation FROM department AS...,Generate SQL query: What are the distinct crea...
9,"SELECT T1.name, T1.num_employees FROM departme...",Generate SQL query: Show the name and number o...
12,SELECT DISTINCT T1.age FROM management AS T2 J...,Generate SQL query: What are the distinct ages...
13,SELECT T3.born_state FROM department AS T1 JOI...,Generate SQL query: List the states where both...
14,"SELECT T1.department_id, T1.name, COUNT(*) FRO...",Generate SQL query: Which department has more ...
...,...,...
4511,SELECT DISTINCT T1.Name FROM singer AS T1 JOIN...,Generate SQL query: Show distinct names of sin...
4512,SELECT T1.Name FROM singer AS T1 JOIN song AS ...,Generate SQL query: Show the names of singers ...
4513,"SELECT T1.Name, SUM(T2.Sales) FROM singer AS T...",Generate SQL query: Show the names of singers ...
4517,SELECT T2.feature_type_name FROM Other_Availab...,Generate SQL query: What is the feature type n...


In [8]:
template = '''Given an input question, first create a syntactically correct SQL query to run, then look at the results of the query and return the answer.\n Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.\nNever query for all columns from a table. You must query only the columns that are needed to answer the question. Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.\nPay attention to use date('now') function to get the current date, if the question involves \"today\".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of the SQLQuery

If someone asks for the art table, they really mean the artworks table.
If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

If the question does not seem related to the database, just return "I don't know" as the answer.

The SQL query should be outputted plainly, do not surround it in quotes or anything else.

Question: {}'''

In [9]:
dsdf_['prompt'] = dsdf_["Instruction"].apply(lambda x: template.format(x))


In [10]:
display(dsdf_), dsdf_.shape


,answer,Instruction,prompt
0,SELECT COUNT(*) FROM head WHERE age > 56,Generate SQL query: How many heads of the depa...,"Given an input question, first create a syntac..."
1,"SELECT name, born_state, age FROM head ORDER B...","Generate SQL query: List the name, born state ...","Given an input question, first create a syntac..."
2,"SELECT creation, name, budget_in_billions FROM...","Generate SQL query: List the creation year, na...","Given an input question, first create a syntac..."
3,"SELECT MAX(budget_in_billions), MIN(budget_in_...",Generate SQL query: What are the maximum and m...,"Given an input question, first create a syntac..."
4,SELECT AVG(num_employees) FROM department WHER...,Generate SQL query: What is the average number...,"Given an input question, first create a syntac..."
...,...,...,...
39336,SELECT MIN(league_cup) FROM table,Generate SQL query: What is the lowest league ...,"Given an input question, first create a syntac..."
39337,SELECT position FROM table WHERE player = Becchio,Generate SQL query: What is the position for t...,"Given an input question, first create a syntac..."
39338,SELECT COUNT(league_cup) FROM table WHERE r = 4,Generate SQL query: How many times was the r 4...,"Given an input question, first create a syntac..."
59542,"SELECT college FROM ""t"" AS able_name_20 WHERE ...",Generate SQL query: What college did the T who...,"Given an input question, first create a syntac..."


(None, (4540, 3))

In [11]:
dsdf_.drop(columns = ['Instruction'], inplace=True)
dsdf_.rename(columns={'answer': 'response'}, inplace=True)
dsdf_ = dsdf_[['prompt', 'response']]
display(dsdf_)

,prompt,response
0,"Given an input question, first create a syntac...",SELECT COUNT(*) FROM head WHERE age > 56
1,"Given an input question, first create a syntac...","SELECT name, born_state, age FROM head ORDER B..."
2,"Given an input question, first create a syntac...","SELECT creation, name, budget_in_billions FROM..."
3,"Given an input question, first create a syntac...","SELECT MAX(budget_in_billions), MIN(budget_in_..."
4,"Given an input question, first create a syntac...",SELECT AVG(num_employees) FROM department WHER...
...,...,...
39336,"Given an input question, first create a syntac...",SELECT MIN(league_cup) FROM table
39337,"Given an input question, first create a syntac...",SELECT position FROM table WHERE player = Becchio
39338,"Given an input question, first create a syntac...",SELECT COUNT(league_cup) FROM table WHERE r = 4
59542,"Given an input question, first create a syntac...","SELECT college FROM ""t"" AS able_name_20 WHERE ..."


In [12]:
dsdf_['prompt'].apply(lambda row: len(row.split())).sort_values()

264     246
1148    246
1664    247
3763    247
906     247
       ... 
2031    287
1971    289
1488    290
2071    290
1114    293
Name: prompt, Length: 4540, dtype: int64

In [13]:
dolly_llama_df = dsdf_.sample(frac=1, random_state=42)
dolly_llama_df.drop_duplicates(inplace=True)
dolly_llama_df.shape
     

(4540, 2)

In [14]:
dolly_llama_df['response'] = dolly_llama_df['response'] +  "\n### End"


In [15]:
dolly_llama_df.drop_duplicates(inplace=True)
display(dolly_llama_df), dolly_llama_df.shape

,prompt,response
1647,"Given an input question, first create a syntac...",SELECT AVG(Years_working) FROM journalist\n###...
1476,"Given an input question, first create a syntac...",SELECT DISTINCT T2.director FROM Rating AS T1 ...
2111,"Given an input question, first create a syntac...",SELECT languages FROM song GROUP BY languages ...
3610,"Given an input question, first create a syntac...","SELECT school, TYPE FROM school WHERE NOT scho..."
691,"Given an input question, first create a syntac...","SELECT guest_first_name, guest_last_name FROM ..."
...,...,...
4426,"Given an input question, first create a syntac...",SELECT Record_Company FROM orchestra GROUP BY ...
466,"Given an input question, first create a syntac...",SELECT T1.Name FROM GENRE AS T1 JOIN TRACK AS ...
3092,"Given an input question, first create a syntac...",SELECT COUNT(*) FROM STUDENT\n### End
3772,"Given an input question, first create a syntac...",SELECT COUNT(*) FROM Staff\n### End


(None, (4540, 2))

In [16]:
from pathlib import Path
Path("dolly_llama_instruct_db.db").touch()

In [17]:
import sqlite3
conn = sqlite3.connect("dolly_llama_instruct_db_v2.db")
c = conn.cursor()


In [18]:
dolly_llama_df.to_sql('dolly_llama_instruct_db_v2', conn, if_exists = 'append', index= False)

4540

In [19]:
c.execute("select * from dolly_llama_instruct_db_v2 limit 3").fetchall()

[('Given an input question, first create a syntactically correct SQL query to run, then look at the results of the query and return the answer.\n Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.\nNever query for all columns from a table. You must query only the columns that are needed to answer the question. Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.\nPay attention to use date(\'now\') function to get the current date, if the question involves "today".\n\nUse the following format:\n\nQuestion: Question here\nSQLQuery: SQL Query to run\nSQLResult: Result of the SQLQuery\n\nIf someone asks for the art table, they really mean the artworks table.\nIf you get an error while 